In [1]:
import pandas as pd
import numpy as np
import re


# 함수 정리

In [2]:
# 패턴을 이용하여 저장 값을 리스트로 되돌리는 함수
def make_ls(c,i): # 인덱스 리스트와 순서를 받음
    matches = re.findall(r"'(.*?)'", c[i])
    return list(matches)

def clear_text(sentence):  # 목차 내용 정제 함수
    pattern = r'[\w가-힣]+'
    extracted = re.findall(pattern, sentence)
    result = ' '.join(extracted)
    return result
    
def delete_text(sentence):
    pattern = r'u3000|u000|u00|uf0ab|u2005|'#목차 더보기'
    result = re.sub(pattern, '', sentence)
    return result



In [3]:
def split_pn2(line): # 페이지 분리 함수 |Ⅸ
    texts = line.split(' ')
    if len(texts) >= 2:
        l = texts[-1]
        ll = texts[-2]
        a = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', l).strip()
        b = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', ll).strip()
        if l.isnumeric() == True and a == '':
            if ll.isnumeric() == True and b == '':
                page = l
                line = ' '.join(line.split(' ')[:-1])
                if len(page) > 3:
                    k = 'pn'
                else:
                    k = 'pn'
            elif ll.isnumeric() == True and b != '':
                page = l
                line = ' '.join(line.split(' ')[:-1])
                if len(page) > 3:
                    k = 'pn'
                else:
                    k = 0
            else:
                page = l
                line = ' '.join(line.split(' ')[:-1])
                if len(page) > 3:
                    k = 'pn'
                else:
                    k = 0
        else:
            page = np.NaN
            k = 0
    else:
        l = texts[-1]
        a = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', l).strip()
        if l.isnumeric() == True and a == '':
            page = l
            line = ' '.join(line.split(' ')[:-1])
            if len(page) > 3:
                k = 'pn'
            else:
                k = 0
        else:
            page = np.NaN
            k = 0
    return line, page, k 

In [4]:
# 구역
def circle(ck_ls,pandan):
    bin_ls = []
    x = -1
    for i in ck_ls:
        if i == pandan[0]:
            x += 1
            bin_ls.append(x)
        else:
            bin_ls.append(x)
    return bin_ls

# 구간별 마지막 뎁스           
def part_check(part, total, ck_ls, bin_ls):
    for o in range(len(part)):
        x = [ 1 for j in range(len(ck_ls)) if bin_ls[j] == o and ck_ls[j] == part[o]]
        if sum(x) <= 1:
            part[o] = total[o][-2]
        else:
            pass
    return part


def study_pandan2(depth):
    study = []
    for i in range(len(depth)):
        if i == 0:
            if depth[i] == depth[i+1]:
                study.append('Y')
            else:
                study.append('N')
        elif i != len(depth)-1:
            if depth[i] == depth[i+1] or depth[i] == depth[i-1]:
                study.append('Y')
            else:
                study.append('N')
        else:
            study.append(study[-1])
    return study

# 뎁스
def assign_depth(lst):
    dep_ls = []
    dep = 1
    depth_map = {}
    for element in lst:
        if element not in depth_map:
            depth_map[element] = dep
            dep += 1
        dep_ls.append(depth_map[element])
    return dep_ls

In [5]:
def classification_roma(line):
    o =''
    test = line.split(' ')
    test = [1 for t in test if re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S','', t) != '']
    if sum(test)> 1:
        o = 'over'
    else:
        o = 0
    return o

def classification_ara(line):
    o =''
    test = line.split(' ')
    test = [1 for t in test if t.isnumeric() == True]
    if sum(test)> 1:
        o = 'over'
    else:
        o = 0
    return o    

def classification_num(line):
    o =''
    if line.count(line[1]) > 1:
        o = 'over'
    else:
        o = 0
    return o    

def classification_ex(line, b):
    o =''
    if line.count(b.group())>1:
        o = 'over'
    else:
        o = 0  
    return o

In [6]:
def classification_list3(text, ex): # 판별 요소 설정 함수
    pd_l,ck_l, ck_ls,pn_l, etc_l = [],[],[],[],[]
    for idx, line in enumerate(text):
        a = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', line).strip()
        # 숫자 판별
        l, page, k= split_pn2(line)
        if line[0].isnumeric() == True: 
            if a != '' and 'roma' not in pd_l: # 로마 숫자
                ck_l.append(l) # 위치 저장
                pn_l.append(page)
                ck_ls.append('roma')
                pd_l.append('roma')
                o = classification_roma(line)
                etc_l.append([o,k])
            elif a != '' and 'roma' in pd_l: # 로마 숫자
                ck_l.append(l) # 위치 저장
                pn_l.append(page)
                ck_ls.append('roma')
                o = classification_roma(line)
                etc_l.append([o,k])
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' not in pd_l :
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append('num')
                pd_l.append('num') # 분류랑 숫자 결합
                o = classification_num(line)
                etc_l.append([o,k])
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' in pd_l:
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append('num')
                o = classification_num(line)
                etc_l.append([o,k])
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' not in pd_l :
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append('ara')
                pd_l.append('ara') # 분류랑 숫자 결합
                o = classification_ara(line)
                etc_l.append([o,k])
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' in pd_l:
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append('ara')
                o = classification_ara(line)
                etc_l.append([o,k])
        elif line[0].isnumeric() != True:   # 포함되는 리스트
            t = re.sub(r'[^a-zA-Z가-힣]','',line.lower())  # 기록 시에는 line을 기록해야 함
            b = ex.match(t)
            if b and b.group() not in pd_l:
                pd_l.append(b.group())
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append(b.group())
                o = classification_ex(line, b)
                etc_l.append([o,k])
            elif b and b.group() in pd_l:
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append(b.group())
                o = classification_ex(line, b)
                etc_l.append([o,k])
                pass
            else:
                pd_l.append('check')
                
    return pd_l, ck_l,ck_ls, pn_l,etc_l

In [7]:
def clear_idx(isbn, tit, text, i, ex):
    code = isbn[i] # isbn, main_title
#     title = tit[i]
#     d1,d2,d3,d4 = 0,0,0,0  # 뎁스 채울 변수(초기화)
    if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
        text.remove('목차 더보기')
    text = [clear_text(r) for r in text] # 이상한 문자 제거
    text = [delete_text(r) for r in text]
    pandan,ck_l, ck_ls, pn_l,etc_l = classification_list3(text,ex)
    if '총론' in pandan and '각론' in pandan:
        pandan.remove('각론')
        ck_ls = ['총론' if c == '각론' else c for c in ck_ls]
    
    if 'check' not in pandan:
        myID = [f'n{f"{i}".zfill(3)}' for i in range(len(ck_ls))] # 바로저장으로 수정+ 전달도 있어야 함
        dep = assign_depth(ck_ls)
        bin_ls = circle(ck_ls,pandan)
        high = assign_highID(myID, dep)#assign_floor_index(myID, dep) # assign_high(ck_ls, myID, pandan) # high_depthID(ck_ls, pandan,myID)
        study = study_pandan2(dep) 
        
        ls = check_string(ck_l)
        t = zero_ls(bin_ls, ls)
        print(t)
        try:
            zero_change(high, dep, study, t)
        except:
            pass
        
        for mi in myID:
            mokchaID.append(mi)
        for ci in ck_l:
            contents.append(ci)
        for pi in pn_l:
            start_pn.append(pi)
        for di in dep:
            depth.append(di)
        for hi in high:
            high_mokchaID.append(hi)
        for si in study:
            learning.append(si)   
        for ei in etc_l:
            etc.append(ei)
        for j in range(len(ck_l)):
            bookID.append(code)
    else:
        bookID.append(code)
#         etc.append('check')
        for j in [mokchaID, high_mokchaID, contents, learning, depth, start_pn, etc]:
            j.append(np.NaN)

In [8]:
def check_string(contents):
    ls = []
    a = re.compile('부록')
    b = re.compile('정답|해설|색인')
    for i,v in enumerate(contents):
        if a.match(v):
            ls.append(i)
        elif b.search(v):
            ls.append(i)
        else:
            pass
        
    return ls


def zero_change(high, dep, study, t):
    if t == 'X':
        pass
    elif len(t)>1:
        for ti in t:
            high[ti] = ''
            dep[ti] = ''
            study[ti] = 'N'
    
    
def zero_ls(circle, ls):
    t = ''
    if len(ls) == 1:
        t = [i for i in range(ls[0],len(circle)+1)]
    elif len(ls) > 1:
        t = []
        for l in ls:
            ck = circle[l]
            zero_plus(circle, l, ck,t)
            
    else:
        t = 'X' # 바꿀 정보 없음
    return t

## zero_ls의 후속함수
def zero_plus(circle, l, ck, t):  
    for i, v in enumerate(circle):
        if i >= l and v == ck:
            t.append(i)
        else:
            pass 

In [9]:
# 상위 뎁스 판단 함수
def assign_highID(myID, dep):
    high = []
    depth_dic = {}
    for i in range(len(dep)):
        if dep[i] == 1:
            high.append(0)
            depth_dic[dep[i]] = myID[i]
        elif dep[i] == 2:
            high.append(depth_dic[1])
            depth_dic[dep[i]] = myID[i]
        elif dep[i] == 3:
            high.append(depth_dic[2])
            depth_dic[dep[i]] = myID[i]
        elif dep[i] == 4:
            high.append(depth_dic[3])
            depth_dic[dep[i]] = myID[i]
        elif dep[i] == 5:
            high.append(depth_dic[4])
            depth_dic[dep[i]] = myID[i]
        else:
            high.append('?')
    return high
            

# 데이터 로딩

In [10]:
# raw data
df1 = pd.read_excel("C://practice_data//new_book_08_08.xlsx")
# # df1 = df1[['main_title','set_Isbn','isbn']]
# df2 = pd.read_excel("C:\kyojeong_data\kj_test2(part).xlsx")

In [11]:
raw_df = df1[['set_isbn','isbn','index']]

In [12]:
raw_df['isbn'] = raw_df['isbn'].apply(lambda x : f'm{x}')

C:\Users\sub computer\AppData\Local\Temp\ipykernel_11128\2874734567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['isbn'] = raw_df['isbn'].apply(lambda x : f'm{x}')


In [13]:
raw_df

,set_isbn,isbn,index
0,9791169660891,m9791169660891,"['""제1편 독해', '제1장 독해의 이론', '01 글의 전개 방식', '02 논..."
1,9791169660891,m9791169660907,NaN
2,9791169660891,m9791169660921,NaN
3,9791169660891,m9791169660914,NaN
4,0,m9791169660549,"['PART 01 문장의 구조, 동사 유형', 'Chapter 01 문장의 구성과 ..."
5,0,m9791169660556,"['PART 01 거시적 읽기', '독해공식 01 두괄식 구조', '독해공식 02 ..."
6,0,m9791160183658,"['PART 01 빈출 보카 + 이디엄 (Day 01-30)', 'DAY 01 (T..."
7,9791169660419,m9791169660419,"['[1권 전근대사]', 'part 1 한국사의 바른 이해', '1 역사의 의미와 ..."
8,9791169660419,m9791169660433,NaN
9,9791169660419,m9791169660426,NaN


In [14]:
df = raw_df.loc[raw_df['set_isbn']== 0,:]

In [15]:
df

,set_isbn,isbn,index
4,0,m9791169660549,"['PART 01 문장의 구조, 동사 유형', 'Chapter 01 문장의 구성과 ..."
5,0,m9791169660556,"['PART 01 거시적 읽기', '독해공식 01 두괄식 구조', '독해공식 02 ..."
6,0,m9791160183658,"['PART 01 빈출 보카 + 이디엄 (Day 01-30)', 'DAY 01 (T..."
10,0,m9791169660440,"['T01 한국사의 바른 이해', 'T02 선사 시대의 전개', 'T03 고조선의 ..."
11,0,m9788952643018,"['P A R T 1 화공양론', 'CHAPTER 01 일반적인 지식이 요구되는 사..."
12,0,m9788952643629,"['PART 1 화공양론', 'CHAPTER 01 일반적인 지식이 요구되는 사항',..."
13,0,m9788952643001,"['P A R T 1 유기공업화학', 'CHAPTER 01 공업화학 기초', 'CH..."
14,0,m9788952643568,"['PART 1 유기공업화학', 'CHAPTER 01 공업화학 기초', 'CHAPT..."
15,0,m9791162404300,"['A 목적 찾기', '[구문 특강] 문장의 형식', '자이 쌤’s Follow M..."
16,0,m9791167224606,"['우리 역사의 시작', '고대 사회의 발전', '정치', '경제', '사회', '..."


In [16]:
df.index = range(len(df))

In [17]:
df

,set_isbn,isbn,index
0,0,m9791169660549,"['PART 01 문장의 구조, 동사 유형', 'Chapter 01 문장의 구성과 ..."
1,0,m9791169660556,"['PART 01 거시적 읽기', '독해공식 01 두괄식 구조', '독해공식 02 ..."
2,0,m9791160183658,"['PART 01 빈출 보카 + 이디엄 (Day 01-30)', 'DAY 01 (T..."
3,0,m9791169660440,"['T01 한국사의 바른 이해', 'T02 선사 시대의 전개', 'T03 고조선의 ..."
4,0,m9788952643018,"['P A R T 1 화공양론', 'CHAPTER 01 일반적인 지식이 요구되는 사..."
5,0,m9788952643629,"['PART 1 화공양론', 'CHAPTER 01 일반적인 지식이 요구되는 사항',..."
6,0,m9788952643001,"['P A R T 1 유기공업화학', 'CHAPTER 01 공업화학 기초', 'CH..."
7,0,m9788952643568,"['PART 1 유기공업화학', 'CHAPTER 01 공업화학 기초', 'CHAPT..."
8,0,m9791162404300,"['A 목적 찾기', '[구문 특강] 문장의 형식', '자이 쌤’s Follow M..."
9,0,m9791167224606,"['우리 역사의 시작', '고대 사회의 발전', '정치', '경제', '사회', '..."


In [18]:
# 변수 설정
isbn, tit, text = df['isbn'], df['set_isbn'], df['index']
# 저장될 리스트
bookID, mokchaID, high_mokchaID, contents, learning, depth,etc= [],[],[],[],[],[],[]
start_pn, end_pn = [],[]

In [153]:
# 지정 구분자 # 시행, 약점, 연습
ex = re.compile("제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|참고|판례|핵심|시행|약점|연습|기출|t|book|part|chapter|day|section|theme|case|keyword|point|q|question|contents")

# 실행

In [20]:
# 실행
for i in range(len(text)):
    try:
        tet = make_ls(text, i)
        clear_idx(isbn, tit, tet, i, ex)
    except:
#         print('i')
        bookID.append(isbn[i])
        for k in [mokchaID, high_mokchaID, contents, learning, depth, start_pn,etc]:
            k.append(np.NaN)

X
[23, 24]
[22, 23, 24, 25, 25]
[12, 13]
X
[31, 32]
X
X
X
X
X


In [21]:
len(learning),len(mokchaID),len(start_pn),len(high_mokchaID)

(341, 341, 341, 341)

In [22]:
data = pd.DataFrame({'txtb_id':bookID,'tcntn_id':mokchaID, 'uppr_tcntn_id':high_mokchaID,'tcntn_trn':etc,\
                   'tcntn_nm':contents, 'std_lnk_yn':learning, 'depth':depth,'start_page':start_pn})
data

,txtb_id,tcntn_id,uppr_tcntn_id,tcntn_trn,tcntn_nm,std_lnk_yn,depth,start_page
0,m9791169660549,n000,0,"[0, 0]",PART 01 문장의 구조 동사 유형,N,1,NaN
1,m9791169660549,n001,n000,"[0, 0]",Chapter 01 문장의 구성과 종류,N,2,NaN
2,m9791169660549,n002,n001,"[0, 0]",Point 01 문장의 구성,Y,3,NaN
3,m9791169660549,n003,n001,"[0, 0]",Point 02 의문문의 어순,Y,3,NaN
4,m9791169660549,n004,n000,"[0, 0]",Chapter 02 동사의 유형,N,2,NaN
...,...,...,...,...,...,...,...,...
336,m9791197461033,n012,0,"[0, 0]",Chapter 13 교원단체 교원노동조합,Y,1,NaN
337,m9791197461033,n013,0,"[0, 0]",Chapter 14 교육환경 보호에 관한 법률,Y,1,NaN
338,m9791197461033,n014,0,"[0, 0]",Chapter 15 학교안전사고 예방 및 보상에 관한 법률,Y,1,NaN
339,m9791197461033,n015,0,"[0, 0]",Chapter 16 학교폭력예방 및 대책에 관한 법률,Y,1,NaN


In [23]:
# set 구분 없는 버전
data.to_excel("C:/Users/sub computer/Desktop/선행목차우선=.xlsx",encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 수작업 목차 정제

In [222]:
tex = re.compile("book|유형|week|제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|참고|판례|핵심|시행|약점|연습|기출|t|book|part|chapter|day|section|theme|case|keyword|point|q|question|contents")

In [229]:
# 초기화
isbn, tit, text = df['isbn'], df['set_isbn'], df['index']

bookID, mokchaID, high_mokchaID, contents, learning, depth,etc= [],[],[],[],[],[],[]
start_pn, end_pn = [],[]

In [207]:
df.loc[df['isbn']=='m9791155062210',:]

,set_isbn,isbn,index
29,0,m9791155062210,['Book 1 진도별 핵심체크 111\n001 S+완전자동사(V1)+전치사+명사 ...


In [220]:
re.findall(r"'(.*?)'", df['index'][28])

['Chapter 01 교육관계법의 체계',
 'Chapter 02 헌법',
 'Chapter 03 교육기본법',
 'Chapter 04 초ㆍ중등교육법',
 'Chapter 05 평생교육법',
 'Chapter 06 고등교육법',
 'Chapter 07 사립학교법',
 'Chapter 08 국가공무원법',
 'Chapter 09 교육공무원법',
 'Chapter 10 교원 등의 연수에 관한 규정',
 'Chapter 11 지방교육자치에 관한 법률',
 'Chapter 12 지방교육재정교부금법',
 'Chapter 13 교원단체, 교원노동조합',
 'Chapter 14 교육환경 보호에 관한 법률',
 'Chapter 15 학교안전사고 예방 및 보상에 관한 법률',
 'Chapter 16 학교폭력예방 및 대책에 관한 법률',
 'Chapter 17 공교육 정상화 촉진 및 선행교육 규제에 관한 특별법',
 '목차 더보기']

In [217]:
df['index'][28]

"['Chapter 01 교육관계법의 체계', 'Chapter 02 헌법', 'Chapter 03 교육기본법', 'Chapter 04 초ㆍ중등교육법', 'Chapter 05 평생교육법', 'Chapter 06 고등교육법', 'Chapter 07 사립학교법', 'Chapter 08 국가공무원법', 'Chapter 09 교육공무원법', 'Chapter 10 교원 등의 연수에 관한 규정', 'Chapter 11 지방교육자치에 관한 법률', 'Chapter 12 지방교육재정교부금법', 'Chapter 13 교원단체, 교원노동조합', 'Chapter 14 교육환경 보호에 관한 법률', 'Chapter 15 학교안전사고 예방 및 보상에 관한 법률', 'Chapter 16 학교폭력예방 및 대책에 관한 법률', 'Chapter 17 공교육 정상화 촉진 및 선행교육 규제에 관한 특별법', '목차 더보기']"

In [236]:
df['index'][29]

"['Book 1 진도별 핵심체크 111\n001 S+완전자동사(V1)+전치사+명사 12\n002 S+완전자동사(V1) 15\n003 S+불완전자동사(V2)+주격보어 18\n004 S+완전타동사(V3)+O 21\n005 유사 의미의 3형식 동사 어법 구분 25\n006 4형식ㆍ5형식 동사로 착각하기 쉬운 3형식 동사 28\n007 S+완전타동사(V3)+재귀대명사(?self) 30\n008 S+수여 동사(V4)+간ㆍ목(사람)+직ㆍ목(사물) 33\n009 사람을 사물 뒤로 쓰지 못하는 수여 동사 35\n010 S+불완전타동사(V5)+O+목적격보어 37\n011 목적격보어 자리에 준동사가 오는 경우 40\n012 유사 형태의 동사 구분 45\n013 유사 의미의 동사 구분 48\n014 타동사의 능동태와 수동태 51\n015 자동사+전치사(타동사구)의 수동태 전환 54\n016 5형식의 수동태 전환 56\n017 수동태로 나타내지 못하는 경우 58\n018 수동태 진행형 / 능동형 수동태 60\n019 타동사+to R/타동사+…ing(동명사) 62\n020 준동사의 명사적 용법 : 진주어와 진목적어 66\n021 전치사의 목적어로 쓰이는 준동사 68\n022 동명사 관용구문 72\n023 부정사의 형용사적 용법 76\n024 분사의 형용사적 용법 : …ing(현재분사) / pp(과거분사) 79\n025 부정사의 부사적 용법 83\n026 분사구문 : 현재분사 …ing를 이용해 절을 구로 줄인 구문 85\n027 준동사의 의미상 주어를 표시하는 방법 87\n028 준동사의 태 : 능동태 준동사 / 수동태 준동사 90\n029 준동사의 시제 : 단순시제 / 완료시제 95\n030 준동사의 부정 : 준동사의 바로 앞에서 98\n031 문두 준동사 구문의 의미상 주어의 일치 101\n032 예외 준동사 구문 103\n033 원형부정사(R)를 쓰는 주요 구문 106\n034 시간ㆍ조건 부사절 속의 시제의 대체 109\n035 시제 부사어구 112\n036 과거완료시제 : h

In [230]:
text = make_ls(text, 28)
text

['Chapter 01 교육관계법의 체계',
 'Chapter 02 헌법',
 'Chapter 03 교육기본법',
 'Chapter 04 초ㆍ중등교육법',
 'Chapter 05 평생교육법',
 'Chapter 06 고등교육법',
 'Chapter 07 사립학교법',
 'Chapter 08 국가공무원법',
 'Chapter 09 교육공무원법',
 'Chapter 10 교원 등의 연수에 관한 규정',
 'Chapter 11 지방교육자치에 관한 법률',
 'Chapter 12 지방교육재정교부금법',
 'Chapter 13 교원단체, 교원노동조합',
 'Chapter 14 교육환경 보호에 관한 법률',
 'Chapter 15 학교안전사고 예방 및 보상에 관한 법률',
 'Chapter 16 학교폭력예방 및 대책에 관한 법률',
 'Chapter 17 공교육 정상화 촉진 및 선행교육 규제에 관한 특별법',
 '목차 더보기']

### 체크 포함

In [231]:
text = df['index'][29].split('\n')
text

["['Book 1 진도별 핵심체크 111",
 '001 S+완전자동사(V1)+전치사+명사 12',
 '002 S+완전자동사(V1) 15',
 '003 S+불완전자동사(V2)+주격보어 18',
 '004 S+완전타동사(V3)+O 21',
 '005 유사 의미의 3형식 동사 어법 구분 25',
 '006 4형식ㆍ5형식 동사로 착각하기 쉬운 3형식 동사 28',
 '007 S+완전타동사(V3)+재귀대명사(?self) 30',
 '008 S+수여 동사(V4)+간ㆍ목(사람)+직ㆍ목(사물) 33',
 '009 사람을 사물 뒤로 쓰지 못하는 수여 동사 35',
 '010 S+불완전타동사(V5)+O+목적격보어 37',
 '011 목적격보어 자리에 준동사가 오는 경우 40',
 '012 유사 형태의 동사 구분 45',
 '013 유사 의미의 동사 구분 48',
 '014 타동사의 능동태와 수동태 51',
 '015 자동사+전치사(타동사구)의 수동태 전환 54',
 '016 5형식의 수동태 전환 56',
 '017 수동태로 나타내지 못하는 경우 58',
 '018 수동태 진행형 / 능동형 수동태 60',
 '019 타동사+to R/타동사+…ing(동명사) 62',
 '020 준동사의 명사적 용법 : 진주어와 진목적어 66',
 '021 전치사의 목적어로 쓰이는 준동사 68',
 '022 동명사 관용구문 72',
 '023 부정사의 형용사적 용법 76',
 '024 분사의 형용사적 용법 : …ing(현재분사) / pp(과거분사) 79',
 '025 부정사의 부사적 용법 83',
 '026 분사구문 : 현재분사 …ing를 이용해 절을 구로 줄인 구문 85',
 '027 준동사의 의미상 주어를 표시하는 방법 87',
 '028 준동사의 태 : 능동태 준동사 / 수동태 준동사 90',
 '029 준동사의 시제 : 단순시제 / 완료시제 95',
 '030 준동사의 부정 : 준동사의 바로 앞에서 98',
 '031 문두 준동사 구문의 의미상 주어의 일치 101',
 '032 예외 준동사

In [232]:
if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
    text.remove('목차 더보기')
    print(0)
text = [clear_text(r) for r in text] # 이상한 문자 제거
text = [delete_text(r) for r in text]
print(1)
pandan,ck_l, ck_ls, pn_l,etc_l = classification_list3(text,tex) # 구분자 체크
print(2,pandan)
if '총론' in pandan and '각론' in pandan:
    pandan.remove('각론')
    ck_ls = ['총론' if c == '각론' else c for c in ck_ls]
    
if 'check' in pandan:
    myID = [f'n{f"{i}".zfill(3)}' for i in range(len(ck_ls))] 
    dep = assign_depth(ck_ls)
    print(3)
    bin_ls = circle(ck_ls,pandan)
    high = assign_highID(myID, dep)#assign_floor_index(myID, dep) # assign_high(ck_ls, myID, pandan) # high_depthID(ck_ls, pandan,myID)
    study = study_pandan2(dep) 
    print(4)
    ls = check_string(ck_l)
    t = zero_ls(bin_ls, ls)
    print(t)
    try:
        zero_change(high, dep, study, t)
    except:
        pass
        
    for mi in myID:
        mokchaID.append(mi)
    for ci in ck_l:
        contents.append(ci)
    for pi in pn_l:
        start_pn.append(pi)
    for di in dep:
        depth.append(di)
    for hi in high:
        high_mokchaID.append(hi)
    for si in study:
        learning.append(si)   
    for ei in etc_l:
        etc.append(ei)
    for j in range(len(ck_l)):
        bookID.append(0)

1


IndexError: string index out of range

In [224]:
test = pd.DataFrame({'bookID':bookID,'mokchaID':mokchaID, 'high_mokchaID':high_mokchaID,'etc':etc, \
                   'contents':contents, 'learning':learning, 'depth':depth,'start_pn':start_pn})
test

,bookID,mokchaID,high_mokchaID,etc,contents,learning,depth,start_pn


### 페이지 분리

In [30]:
def split_pn3(line): # 페이지 분리 함수 |Ⅸ
    texts = line.split('ㆍ')
    if len(texts) >= 2:
        l = texts[-1]
        ll = texts[-2]
        a = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', l).strip()
        b = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', ll).strip()
        if l.isnumeric() == True and a == '':
            if ll.isnumeric() == True and b == '':
                page = l
                line = ' '.join(line.split(' ')[:-1])
                if len(page) > 3:
                    k = 'pn'
                else:
                    k = 'pn'
            elif ll.isnumeric() == True and b != '':
                page = l
                line = ' '.join(line.split(' ')[:-1])
                if len(page) > 3:
                    k = 'pn'
                else:
                    k = 0
            else:
                page = l
                line = ' '.join(line.split(' ')[:-1])
                if len(page) > 3:
                    k = 'pn'
                else:
                    k = 0
        else:
            page = np.NaN
            k = 0
    else:
        l = texts[-1]
        a = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', l).strip()
        if l.isnumeric() == True and a == '':
            page = l
            line = ' '.join(line.split(' ')[:-1])
            if len(page) > 3:
                k = 'pn'
            else:
                k = 0
        else:
            page = np.NaN
            k = 0
    return line, page, k 

In [171]:
line, page = [],[]
if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
    text.remove('목차 더보기')
    print(0)
# text = [clear_text(r) for r in text] # 이상한 문자 제거
# text = [delete_text(r) for r in text]
for t in text:
    l, pn, k= split_pn2(t)
    line.append(l)
    page.append(pn)


In [172]:
test = pd.DataFrame({'line':line,'page':page})
test

,line,page
0,[기본편],NaN
1,제1장 내용편,NaN
2,제2장 운문 표현편,NaN
3,제3장 산문 표현편,NaN
4,[문제편],NaN
5,제1장 현대시,NaN
6,제2장 고전시가,NaN
7,제3장 현대소설,NaN
8,제4장 고전산문,NaN
9,"제5장 수필,극",NaN


### 수작업 내용 저장

In [173]:
test.to_excel("C:/Users/sub computer/Desktop/선정목차++.xlsx",encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 개별권들 잘 작업 되었는지 확인

In [204]:
df_isbn = df['isbn'].unique().tolist()
df_isbn

['m9791169660549',
 'm9791169660556',
 'm9791160183658',
 'm9791169660440',
 'm9788952643018',
 'm9788952643629',
 'm9788952643001',
 'm9788952643568',
 'm9791162404300',
 'm9791167224606',
 'm9791167224569',
 'm9791192845470',
 'm9791192845487',
 'm9791138328234',
 'm9791165769277',
 'm9791166626135',
 'm9791189122348',
 'm9791189122409',
 'm9791165768454',
 'm9791138328258',
 'm9791138328210',
 'm9791138327909',
 'm9791165769086',
 'm9791192083490',
 'm9791169660945',
 'm9791198258502',
 'm9791192083483',
 'm9791197461040',
 'm9791197461033',
 'm9791155062210']

In [199]:
check = pd.read_excel("C:\practice_data\선행목차우선=.xlsx")
check

,txtb_id,tcntn_id,uppr_tcntn_id,Unnamed: 3,tcntn_nm,std_lnk_yn,depth,start_page,Unnamed: 8
0,m9791169660549,n000,NaN,NaN,PART 01 문장의 구조 동사 유형,N,1.0,NaN,NaN
1,m9791169660549,n001,n000,NaN,Chapter 01 문장의 구성과 종류,N,2.0,NaN,NaN
2,m9791169660549,n002,n001,NaN,Point 01 문장의 구성,Y,3.0,NaN,NaN
3,m9791169660549,n003,n001,NaN,Point 02 의문문의 어순,Y,3.0,NaN,NaN
4,m9791169660549,n004,n000,NaN,Chapter 02 동사의 유형,N,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1435,m9791192083483,n005,n004,NaN,제1장 현대시,Y,2.0,NaN,NaN
1436,m9791192083483,n006,n004,NaN,제2장 고전시가,Y,2.0,NaN,NaN
1437,m9791192083483,n007,n004,NaN,제3장 현대소설,Y,2.0,NaN,NaN
1438,m9791192083483,n008,n004,NaN,제4장 고전산문,Y,2.0,NaN,NaN


In [202]:
check_isbn = check['txtb_id'].unique().tolist()
len(check_isbn)

29

In [205]:
no_ls = []
for di in df_isbn:
    if di not in check_isbn:
        no_ls.append(di)
no_ls

['m9791155062210']

# 세트권 작업

In [178]:
set_df = raw_df.loc[raw_df['set_isbn']!=0,:]
set_df

,set_isbn,isbn,index
0,9791169660891,m9791169660891,"['""제1편 독해', '제1장 독해의 이론', '01 글의 전개 방식', '02 논..."
1,9791169660891,m9791169660907,NaN
2,9791169660891,m9791169660921,NaN
3,9791169660891,m9791169660914,NaN
7,9791169660419,m9791169660419,"['[1권 전근대사]', 'part 1 한국사의 바른 이해', '1 역사의 의미와 ..."
8,9791169660419,m9791169660433,NaN
9,9791169660419,m9791169660426,NaN
18,9791160183627,m9791160183627,"['〈제1권〉', '제1편 행정법통론', '제2편 행정작용법', '찾아보기(Inde..."
19,9791160183627,m9791160183634,NaN
20,9791160183627,m9791160183641,NaN


In [183]:
# 초기화
isbn, tit, text = set_df['isbn'], set_df['set_isbn'], set_df['index']

bookID, mokchaID, high_mokchaID, contents, learning, depth,etc= [],[],[],[],[],[],[]
start_pn, end_pn = [],[]
set_df.loc[set_df['isbn']=='m9791169660891',:]

,set_isbn,isbn,index
0,9791169660891,m9791169660891,"['""제1편 독해', '제1장 독해의 이론', '01 글의 전개 방식', '02 논..."


In [184]:
text = make_ls(text, 0)
text

['"제1편 독해',
 '제1장 독해의 이론',
 '01 글의 전개 방식',
 '02 논증과 추론',
 '03 쓰기ㆍ말하기',
 '제2장 독해의 기술과 유형',
 '[한눈에 보기] 독해의 힘을 길러 주는 필수 훈련법',
 '01 독해의 기술',
 '02 주요 문제 유형 분석',
 '제2편 현대 문학',
 '제1장 문학 일반론',
 '01 문학 이론',
 '02 다양한 문학적 기법',
 '개념 완성',
 '제2장 문학의 주요 갈래',
 '[한눈에 들어오는] 현대 문학사',
 '01 시의 이해',
 '02 소설의 이해',
 '03 희곡과 시나리오의 이해',
 '04 수필의 이해',
 '제3편 고전 문학',
 '제1장 고전 문학의 이해',
 '[한눈에 들어오는] 고전 문학사',
 '01 고대의 문학',
 '02 고려 시대의 문학',
 '03 조선 시대의 문학',
 '개념 완성',
 '제4편 국어 문법',
 '제1장 국어 문법의 이해',
 '01 음운론',
 '개념 완성',
 '02 형태론',
 '개념 완성',
 '03 통사론',
 '개념 완성',
 '04 의미론',
 '개념 완성',
 '05 담화론',
 '개념 완성',
 '제5편 국어 규범',
 '제1장 올바른 국어 생활의 여러 영역',
 '01 한글 맞춤법',
 '개념 완성',
 '02 표준어 규정',
 '03 표준 발음법',
 '개념 완성',
 '04 문장 부호론',
 '05 로마자 표기법',
 '개념 완성',
 '06 외래어 표기법',
 '개념 완성',
 '제6편 고전 문법',
 '제1장 고전 문법의 이해',
 '[한눈에 들어오는] 국어의 역사',
 '01 고전 문법',
 '개념 완성',
 '〈책 속의 책〉',
 '공무원 기출 어휘 및 한자 · 한자 성어 핵심 정리"',
 '목차 더보기']